**============= Project 2 - Datasets for downstream analysis work =============**

IS362 - Kelvin Rodriguez

**_Dataset #1 - Lottery Powerball Winning Numbers: Beginning 2010_**

In [11]:
import pandas as pd

# Load data from the remote CSV file
df = pd.read_csv("https://raw.githubusercontent.com/KingScanor/IS362-Project_2/refs/heads/main/Lottery_Powerball_Winning_Numbers__Beginning_2010.csv")

# Split 'Winning Numbers' string into separate columns for each ball
winning_numbers = df['Winning Numbers'].str.split(' ', expand=True)

# Rename the new columns
winning_numbers.columns = ['Ball 1', 'Ball 2', 'Ball 3', 'Ball 4', 'Ball 5', 'Bonus Ball']

# Drop the original combined column
df = df.drop('Winning Numbers', axis=1)

# Add the new ball columns to the main DataFrame
df = pd.concat([df, winning_numbers], axis=1)

# Convert 'Draw Date' column to datetime objects
df['Draw Date'] = pd.to_datetime(df['Draw Date'], format='%m/%d/%Y')

# Filter the DataFrame to include only draws from the year 2020
df_2020 = df[df['Draw Date'].dt.year == 2020].copy()

# Convert the 'Ball 1' column to an integer type.
# It also allows for numerical operations.
df_2020['Ball 1'] = df_2020['Ball 1'].astype(int)

# Sort the 2020 data by 'Draw Date'
df_2020_sorted = df_2020.sort_values(by='Draw Date', ascending=True)

# Print a formatted header
print("\n" + "="*57)
print('--- Lottery draws for the year 2020 (Sorted by date)')
print("="*57)

# Print the final sorted DataFrame without the index
print(df_2020_sorted.to_string(index=False))

# .value_counts() counts the frequency of each unique number
# .nlargest(5) selects the top 5 by count
top_5_ball_1_2020 = df_2020['Ball 1'].value_counts().nlargest(5)

# 7. Print the result
print("\n" + "="*50)
print("--- Top 5 Most Common 'Ball 1' Numbers in 2020 ---")
print("="*50)
print(top_5_ball_1_2020)


--- Lottery draws for the year 2020 (Sorted by date)
 Draw Date  Multiplier  Ball 1 Ball 2 Ball 3 Ball 4 Ball 5 Bonus Ball
2020-01-01         2.0      49     53     57     59     62         26
2020-01-04         2.0       1     11     21     25     54         07
2020-01-08         4.0       2     04     07     43     56         22
2020-01-11         2.0       3     21     23     31     59         03
2020-01-15         2.0      39     41     53     55     68         19
2020-01-18         2.0      20     24     38     56     68         18
2020-01-22         3.0      11     33     44     59     67         08
2020-01-25         2.0       2     09     17     36     67         18
2020-01-29         2.0       9     12     15     31     60         02
2020-02-01         4.0      12     33     54     57     60         13
2020-02-05         3.0      23     30     35     41     57         02
2020-02-08         2.0      35     49     50     59     66         06
2020-02-12         2.0      14     4

**Dataset #1 Analysis**

The Winning Numbers column was a space-separated string that needed to be cleaned up. It was broken into six columns: Ball 1 to Ball 5 and Bonus Ball. I took out the original column and put the divided columns back together in the DataFrame. To make filtering and sorting easier, the Draw Date was turned into a pandas datetime object. After that, the dataset was filtered to only keep draws from 2020, which made a more focused DataFrame called df_2020. Finally, the data type in the Ball 1 column was changed from string to integer to be sure that counting and analysis were correct. The analysis I implemented looked at how often numbers were in the "Ball 1" position during the 2020 drawings. I used the pandas methods .value_counts() to count how many times each unique number appeared and .nlargest(5) to get the five most common values in that place. The output shows how many times these popular "Ball 1" numbers came up in 2020. The analysis of 2020 Powerball draws illustrates the frequency distribution of the first ball drawn. The Powerball game is based on chance, but this analysis finds the five numbers that most often showed up in the "Ball 1" slot. But it's important to remember that in a random lottery, prior results don't affect future results because each draw is separate. It was very interesting to play with this kind of data. This exercise shows how to structure and clean lottery data and do basic frequency analysis, which is an important part of statistical research.


**_Dataset #2 - Climate Comparison Chart of Boston, USA vs New York City, USA_**

In [21]:
import pandas as pd

# Define the URLs for the two datasets (snowfall and temperature)
snow_url = 'https://raw.githubusercontent.com/KingScanor/IS362-Project_2/refs/heads/main/Snow%20(mm)%20-%20Boston%20vs%20NYC.csv'
temp_url = 'https://raw.githubusercontent.com/KingScanor/IS362-Project_2/refs/heads/main/Temperature%20%C2%B0C%20-%20Boston%20vs%20NYC.csv'

# Define a function to load data, convert it from wide to long (tidy) format, and add a metric label
def tidy_and_label_data(url: str, metric_name: str) -> pd.DataFrame:

    try:
        df =pd.read_csv(url) # Attempt to load the CSV data from the URL
    except Exception as e:
        print(f"Error loading {metric_name} data: {e}") # Print error if loading fails
        return pd.DataFrame() # Return an empty DataFrame on error

# Use pd.melt to convert the DataFrame from wide (Boston, NYC columns) to long (tidy) format    
    df_tidy = df.melt(
        id_vars=['Month'],# Keep 'Month' as an identifier
        value_vars=['Boston', 'New York City'],# Columns to melt (city values)
        var_name='City',# New column for City names
        value_name='Value' # New column for the corresponding numerical value
    )

    df_tidy['Metric'] = metric_name # Add a new column to label the data (e.g., 'Snowfall (mm)')

    return df_tidy # Return the cleaned, tidy DataFrame

# Process the Snowfall data
df_snow_tidy = tidy_and_label_data(snow_url, 'Snowfall (mm)')

# Process the Temperature data
df_temp_tidy = tidy_and_label_data(temp_url, 'Temperature (°C)')

# Combine the tidy Snowfall and Temperature DataFrames vertically
df_master_tidy = pd.concat([df_snow_tidy, df_temp_tidy], ignore_index=True)

# Print the final combined dataset for inspection
print("\n" + "="*28)
print("--- Final Combined Data ---")
print("="*28)
print(df_master_tidy.head(49))

# --- Analysis: Find Max Snowfall ---

# Filter the master DataFrame to get only Snowfall data
df_snow = df_master_tidy[df_master_tidy['Metric'] == 'Snowfall (mm)']

# Find the index of the maximum 'Value' (snowfall) for each 'City'
idx_snow = df_snow.groupby('City')['Value'].idxmax()

# Use the found indices to retrieve the full rows corresponding to the max snowfall
result_snow = df_snow.loc[idx_snow]

# --- Analysis: Find Min Temperature (Coldest Month) ---

# Filter the master DataFrame to get only Temperature data
df_temp = df_master_tidy[df_master_tidy['Metric'] == 'Temperature (°C)']

# Find the index of the minimum 'Value' (temperature) for each 'City'
idx_temp = df_temp.groupby('City')['Value'].idxmin()

# Use the found indices to retrieve the full rows corresponding to the coldest month
result_temp = df_temp.loc[idx_temp]

# Print the maximum snowfall results
print("\n" + "="*21)
print("--- Most Snowfall ---")
print("="*21)
print(result_snow[['City', 'Month', 'Value']])

# Print the minimum temperature (coldest month) results
print("\n" + "="*21)
print("--- Coldest Month ---")
print("="*21)
print(result_temp[['City', 'Month', 'Value']])


--- Final Combined Data ---
   Month           City  Value            Metric
0    Jan         Boston   0.77     Snowfall (mm)
1    Feb         Boston   1.32     Snowfall (mm)
2    Mar         Boston   0.76     Snowfall (mm)
3    Apr         Boston   0.19     Snowfall (mm)
4    May         Boston   0.00     Snowfall (mm)
5    Jun         Boston   0.00     Snowfall (mm)
6    Jul         Boston   0.00     Snowfall (mm)
7    Aug         Boston   0.00     Snowfall (mm)
8    Sep         Boston   0.00     Snowfall (mm)
9    Oct         Boston   0.02     Snowfall (mm)
10   Nov         Boston   0.10     Snowfall (mm)
11   Dec         Boston   0.44     Snowfall (mm)
12   Jan  New York City   0.48     Snowfall (mm)
13   Feb  New York City   0.85     Snowfall (mm)
14   Mar  New York City   0.61     Snowfall (mm)
15   Apr  New York City   0.04     Snowfall (mm)
16   May  New York City   0.00     Snowfall (mm)
17   Jun  New York City   0.00     Snowfall (mm)
18   Jul  New York City   0.00     Snowf

**Dataset #2 Analysis**

To clean up the data, I had to combine two independent datasets that showed the average monthly snowfall and temperature in Boston and New York City.  The goal of the cleanup was to make the DataFrame tidy enough for analysis.  The pd.melt() function was used to change the data from wide to long format. This made three columns: Month, City, and Value.  Before putting the datasets together, a new column named Metric was created to show what kind of measurement it was.  Finally, the two tidy DataFrames were combined into a master DataFrame, which made it possible to see all of the observations for both cities and metrics in one place.  The analysis of extreme weather measures looked at the months in Boston and New York City that had the most snow and the coldest temperatures.  To find the heaviest snowfall for both cities, the data was filtered to only include "Snowfall (mm)" values. Then, the largest value was found using the .groupby('City')['Value'].idxmax() function.  For temperature, on the other hand, the study employed idxmin() to discover the month with the lowest temperature.  This approach correctly found the month with the lowest temperature in each city.  This whole process shows how to prepare data quickly using the tidy data philosophy in pandas, which makes it easy to filter and do descriptive statistical analysis on many metrics and categories.



**_Dataset #3 - UN Forecast 2024_**

In [31]:
import pandas as pd

# Define the URL for the raw population forecast data
url = "https://raw.githubusercontent.com/KingScanor/IS362-Project_2/refs/heads/main/UN%20Forecast%202024.csv"

try:
    df_UN_Forecast_2024 = pd.read_csv(url) # Attempt to read the CSV into a DataFrame
    
     # Display the first few rows of the raw data
    print("\n" + "="*30)
    print("--- UN Forecast 2024 ---")
    print("="*30)
    print(df_UN_Forecast_2024.head(10))
    print(f"\nTotal rows for UN Forecast 2024: {len(df_UN_Forecast_2024)}")
    
except Exception as e:
    print(f"Error Loading CSV: {e}") # Handle case where CSV loading fails

# Convert the DataFrame from wide format (years as columns) to long/tidy format
df_organized = pd.melt(
    df_UN_Forecast_2024, # The input DataFrame
    id_vars=id_cols, # Columns to keep fixed (Country/Region, Subregion, Region)
    value_vars=year_cols, # Columns to melt (all the year columns)
    var_name='Year', # New column to hold the original column names (the years)
    value_name='Population' # New column to hold the values (the population numbers)
)

# Clean the 'Population' column: Convert to string, remove commas (thousands separator)
df_organized['Population'] = (
    df_organized['Population']
    .astype(str)
    .str.replace(',', '', regex=False)
)

# Convert 'Population' to numeric, handling errors (like non-numeric strings) by setting them to NaN, then convert to integer type
df_organized['Population'] = pd.to_numeric(df_organized['Population'], errors='coerce').astype('Int64')

# Convert 'Year' column from string (from melt operation) to integer type
df_organized['Year'] = pd.to_numeric(df_organized['Year']).astype(int)

# Remove any rows where the 'Population' value is missing (NaN) after cleaning/conversion
df_organized = df_organized.dropna(subset=['Population'])

# Display the final cleaned and organized data structure
print("\n" + "="*57)
print("--- Final Organized Data (Country, Year, Population) ---")
print("="*57)
print(df_organized.head(10))
print(f"\nTotal rows in organized data: {len(df_organized)}")

# Filter the organized DataFrame to include only the year 2030
df_2030 = df_organized[df_organized['Year'] == 2030]

# Sort the 2030 data by 'Population' in descending order (highest first)
df_most_populated_2030 = df_2030.sort_values(
    by='Population',
    ascending=False
)

# Print the top 10 most populated countries for the year 2030
print("\n" + "="*50)
print("--- Top 10 Most Populated Countries in 2030 ---")
print("="*50)
print (df_most_populated_2030.head(10).reset_index(drop=True))
print(f"\nTotal rows for 2030: {len(df_2030)}")


--- UN Forecast 2024 ---
                Country        2024        2030        2050         2100
0           Afghanistan  42,647,492  50,039,402  76,885,134  130,216,739
1               Albania   2,791,765   2,671,885   2,240,166    1,184,997
2               Algeria  46,814,308  50,154,166  59,565,554   64,487,527
3        American Samoa      46,765      42,958      37,545       32,293
4               Andorra      81,938      85,682      82,195       47,222
5                Angola  37,885,849  45,160,458  74,295,394  150,045,574
6              Anguilla      14,598      14,977      14,552        9,309
7   Antigua and Barbuda      93,772      96,000      95,055       67,975
8             Argentina  45,696,159  46,585,022  48,308,944   38,255,990
9               Armenia   2,973,840   2,851,291   2,495,207    1,692,198

Total rows for UN Forecast 2024: 237

--- Final Organized Data (Country, Year, Population) ---
                Country  Year  Population
0           Afghanistan  2024    

**Dataset #3 Analysis**

This code's goal is to clean, restructure, and analyze the UN World Population Forecast data by changing it from a wide format (with years as columns) to a tidy long format (with one nation per row every year). You start by putting the CSV file into a DataFrame and looking at how it is set up. Then you use the pd.melt() function to change the shape of the data. Key identifiers, such as "Country/Region," stay the same, but year columns are combined into a Year and Population format. When cleaning data, you have to deal with missing data and change Population values from strings to numbers. Lastly, entries that aren't numbers are deleted so that only legitimate numeric forecasts stay. The analysis goes into detail about how to get and rank demographic data for 2030. By filtering the df_organized DataFrame to include just forecast data for that year, you can make a DataFrame called df_2030. The data is then sorted by the Population column in reverse order, which gives us a list of the Top 10 Most Populated Countries in 2030 based on UN forecasts. This approach shows how to efficiently handle large datasets to answer specific forecasting questions.

**References**

ClimeChart.com & ClimeChart.com - Michael Caviglia. (n.d.). Boston vs New York City Climate Chart | Weather Overview of Boston, USA and New York City, USA. https://www.climechart.com/en/climate-compare/boston/united-states-of-america/new-york-city/united-states-of-america 

Compare text and find differences online or offline - Diffchecker. (n.d.). https://www.diffchecker.com/text-compare/ 

Data.gov. (2025, October 4). State of New York - Lottery Powerball winning numbers: Beginning 2010. https://catalog.data.gov/dataset/lottery-powerball-winning-numbers-beginning-2010 

Tidy data in Python · Jean-Nicholas Hould. (n.d.). https://www.jeannicholashould.com/tidy-data-in-python.html 

User Guide — pandas 2.3.3 documentation. (n.d.). https://pandas.pydata.org/docs/user_guide/index.html Wikipedia contributors. (2025, July 12). List of countries by past and projected future population. 

Wikipedia. https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_future_population